In [1]:
import torch
import pandas as pd
import csv

In [2]:
# Dataset config
TIME_BEG = 1451606400 # = year 2016
TIME_INTERVAL = 60 # minutes
NUM_ENCODE_TIME_STEPS = 48
NUM_DECODE_TIME_STEPS = 5

In [3]:
prices = None

with open('kaggle_1min.csv', 'r') as f:
    reader = csv.reader(f)
    prices = list(reader)

In [4]:
# Process with Panda dataframe
prices_df = pd.DataFrame(prices[1:], columns=prices[0])

print("Original dataframe: ")
print(prices_df)

# Want Volume Weighted Average Price (VWAP) since TIME_BEG
prices_df = prices_df.drop(columns=["Open", "High", "Low", "Close", "Volume_(Currency)"])
prices_df = prices_df.astype(float)
prices_df = prices_df.fillna(0) # Replace NaN with 0 (time where trade volume = 0)
prices_df = prices_df.loc[prices_df["Timestamp"] >= TIME_BEG] # Cut off from desired begin time
prices_df = prices_df.reset_index(drop=True)

print("Processed dataframe: ")
print(prices_df)

Original dataframe: 
          Timestamp      Open      High       Low     Close Volume_(BTC)  \
0        1325317920      4.39      4.39      4.39      4.39   0.45558087   
1        1325317980       NaN       NaN       NaN       NaN          NaN   
2        1325318040       NaN       NaN       NaN       NaN          NaN   
3        1325318100       NaN       NaN       NaN       NaN          NaN   
4        1325318160       NaN       NaN       NaN       NaN          NaN   
...             ...       ...       ...       ...       ...          ...   
4727772  1609372560  28801.47  28829.42  28785.64  28829.42   0.96522104   
4727773  1609372620  28829.42   28863.9  28829.42  28857.06   2.36883117   
4727774  1609372680  28850.49  28900.52  28850.49  28882.82   2.46658976   
4727775  1609372740  28910.54  28911.52   28867.6   28881.3     7.332773   
4727776  1609372800  28893.21  28928.49  28893.21  28928.49   5.75767938   

        Volume_(Currency) Weighted_Price  
0            2.00000001

In [5]:
# Summarize prices by each TIME_INTERVAL using WVAP calculations
prices_tensor = torch.from_numpy(prices_df.values)
shape = (prices_tensor.shape[0] // TIME_INTERVAL, 2)
condensed_prices_tensor = torch.empty(shape)

print("Original price data shape: ", prices_tensor.shape)
print("Time condensed price data shape: ", condensed_prices_tensor.shape)

j = 0
for i in range(0, prices_tensor.shape[0] - TIME_INTERVAL, TIME_INTERVAL):
    condensed_prices_tensor[j, 0] = prices_tensor[i, 0] # Time stamp

    # Calculate WVAP
    volume_weighted_sum = torch.sum(prices_tensor[i:i+TIME_INTERVAL, 1] * prices_tensor[i:i+TIME_INTERVAL, 2])
    total_volume = torch.sum(prices_tensor[i:i+TIME_INTERVAL, 1])
    wvap = volume_weighted_sum / total_volume

    # If no trade in this time interval use the price in the previous one
    # Here assume that first TIME_INTERVAL since TIME_BEG has > 0 trade volume
    if total_volume == 0:
        condensed_prices_tensor[j, 1] = condensed_prices_tensor[j-1, 1]
    else:
        condensed_prices_tensor[j, 1] = wvap
    
    j += 1

print("Condensed price data: \n", condensed_prices_tensor)

Original price data shape:  torch.Size([2629441, 3])
Time condensed price data shape:  torch.Size([43824, 2])
Condensed price data: 
 tensor([[1.4516e+09, 4.3146e+02],
        [1.4516e+09, 4.3027e+02],
        [1.4516e+09, 4.3068e+02],
        ...,
        [1.6094e+09, 2.8804e+04],
        [1.6094e+09, 2.8800e+04],
        [1.6094e+09, 2.8817e+04]])


In [6]:
# Make input time serie data and target time serie data tensor 
X_shape = (condensed_prices_tensor.shape[0] - NUM_ENCODE_TIME_STEPS - NUM_DECODE_TIME_STEPS, NUM_ENCODE_TIME_STEPS)
y_shape = (condensed_prices_tensor.shape[0] - NUM_ENCODE_TIME_STEPS - NUM_DECODE_TIME_STEPS, NUM_DECODE_TIME_STEPS)

print("Encode time serie data shape:", X_shape)
print("Decode time serie data shape:", y_shape)

X = torch.empty(X_shape)
y = torch.empty(y_shape)

# # Raw price
# for i in range(X.shape[0]):
#     input_end_i = i + NUM_ENCODE_TIME_STEPS
#     target_end_i = input_end_i + NUM_DECODE_TIME_STEPS
#     X[i] = condensed_prices_tensor[i:input_end_i, 1]
#     y[i] = condensed_prices_tensor[input_end_i:target_end_i, 1]

# Growth from last time step (more numerically stable for the neural net)
for i in range(X.shape[0]):
    serie_end_i = i + NUM_ENCODE_TIME_STEPS + NUM_DECODE_TIME_STEPS
    time_serie = (condensed_prices_tensor[i+1:serie_end_i+1, 1] - condensed_prices_tensor[i:serie_end_i, 1]) / condensed_prices_tensor[i:serie_end_i, 1]
    X[i] = time_serie[:NUM_ENCODE_TIME_STEPS]
    y[i] = time_serie[NUM_ENCODE_TIME_STEPS:]

print("Training X:\n", X)
print("Training y:\n", y)

Encode time serie data shape: (43771, 48)
Decode time serie data shape: (43771, 5)
Training X:
 tensor([[-2.7632e-03,  9.5637e-04,  4.1267e-03,  ..., -1.3626e-03,
         -8.3544e-04,  8.8970e-04],
        [ 9.5637e-04,  4.1267e-03,  4.4184e-03,  ..., -8.3544e-04,
          8.8970e-04, -5.3603e-03],
        [ 4.1267e-03,  4.4184e-03,  2.1721e-03,  ...,  8.8970e-04,
         -5.3603e-03, -1.7559e-03],
        ...,
        [ 3.5812e-03, -8.7739e-04,  1.3712e-05,  ...,  2.6845e-03,
          3.9756e-03, -2.1672e-03],
        [-8.7739e-04,  1.3712e-05, -7.7917e-03,  ...,  3.9756e-03,
         -2.1672e-03,  4.1753e-03],
        [ 1.3712e-05, -7.7917e-03, -1.9747e-03,  ..., -2.1672e-03,
          4.1753e-03,  6.6546e-03]])
Training y:
 tensor([[-0.0054, -0.0018,  0.0029, -0.0003, -0.0007],
        [-0.0018,  0.0029, -0.0003, -0.0007, -0.0015],
        [ 0.0029, -0.0003, -0.0007, -0.0015,  0.0001],
        ...,
        [ 0.0042,  0.0067,  0.0125,  0.0021, -0.0003],
        [ 0.0067,  0.0125,

In [7]:

torch.save(X, "./../X.pt")
torch.save(y, "./../y.pt")

In [8]:
X = torch.load("./../X.pt")
y = torch.load("./../y.pt")
print(X)
print(y)

tensor([[-2.7632e-03,  9.5637e-04,  4.1267e-03,  ..., -1.3626e-03,
         -8.3544e-04,  8.8970e-04],
        [ 9.5637e-04,  4.1267e-03,  4.4184e-03,  ..., -8.3544e-04,
          8.8970e-04, -5.3603e-03],
        [ 4.1267e-03,  4.4184e-03,  2.1721e-03,  ...,  8.8970e-04,
         -5.3603e-03, -1.7559e-03],
        ...,
        [ 3.5812e-03, -8.7739e-04,  1.3712e-05,  ...,  2.6845e-03,
          3.9756e-03, -2.1672e-03],
        [-8.7739e-04,  1.3712e-05, -7.7917e-03,  ...,  3.9756e-03,
         -2.1672e-03,  4.1753e-03],
        [ 1.3712e-05, -7.7917e-03, -1.9747e-03,  ..., -2.1672e-03,
          4.1753e-03,  6.6546e-03]])
tensor([[-0.0054, -0.0018,  0.0029, -0.0003, -0.0007],
        [-0.0018,  0.0029, -0.0003, -0.0007, -0.0015],
        [ 0.0029, -0.0003, -0.0007, -0.0015,  0.0001],
        ...,
        [ 0.0042,  0.0067,  0.0125,  0.0021, -0.0003],
        [ 0.0067,  0.0125,  0.0021, -0.0003, -0.0001],
        [ 0.0125,  0.0021, -0.0003, -0.0001,  0.0006]])


In [9]:
print(X.shape)
print(torch.mean(X))
print("Maximum price change in", TIME_INTERVAL, "minutes", torch.max(X))
print("Minimum price change in", TIME_INTERVAL, "minutes", torch.min(X))

torch.Size([43771, 48])
tensor(0.0001)
Maximum price change in 60 minutes tensor(0.1391)
Minimum price change in 60 minutes tensor(-0.1557)


In [10]:
# Checking for nan data
assert(torch.sum(X != X) == 0)
assert(torch.sum(y != y) == 0)